In [2]:
import pandas as pd
import os 
import sys
import glob 

current_path = notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)
from build_inputs.load_raw_data import load_CRITER

ROOT_DATA = '../../../../data/rrochas'
AGG_DATA_PATH = f"{ROOT_DATA}/prediction_validation/agg_data"


## Open validation_individuelle:
files = [
    'subway_indiv_3min','tramway_indiv_3min','bus_indiv_3min','vaf_indiv_3min',
    'subway_indiv_5min','tramway_indiv_5min','bus_indiv_5min','vaf_indiv_5min',
    'subway_indiv_6min','tramway_indiv_6min','bus_indiv_6min','vaf_indiv_6min',
    'subway_indiv_10min','tramway_indiv_10min','bus_indiv_10min','vaf_indiv_10min',
    'subway_indiv_15min','tramway_indiv_15min','bus_indiv_15min','vaf_indiv_15min'
]

df_subway_3min = pd.read_csv(f"{AGG_DATA_PATH}/validation_individuelle/{files[0]}/{files[0]}.csv")
df_tramway_15min = pd.read_csv(f"{AGG_DATA_PATH}/validation_individuelle/{files[-3]}/{files[-3]}.csv")
df_bus_15min = pd.read_csv(f"{AGG_DATA_PATH}/validation_individuelle/{files[-2]}/{files[-2]}.csv")
## ----

## Open velov:
direction = 'emitted' #'attracted'
temporal_agg = '30min' #'2min','3min','5min', '6min', '10min', '15min','30min','1H'
df_velov_30min = pd.read_csv(f"{AGG_DATA_PATH}/velov/velov_{direction}_by_station{temporal_agg}.csv")
## ----


## Open CRITER:
from build_inputs.load_raw_data import load_CRITER
CRITER_PATH = f"{ROOT_DATA}/raw_data/Comptages_Velo_Routier/CRITER"
years = [2019,2020]
list_files = sorted(glob.glob(os.path.join(f'{CRITER_PATH}/6 min {years[0]}', "*.txt"))) + sorted(glob.glob(os.path.join(f'{CRITER_PATH}/6 min {years[1]}', "*.txt")))
oct_2019_mar_2020 = list_files[9:15]
df_criter_6min_october = load_CRITER(oct_2019_mar_2020[0])
## ----

/home/rrochas/prediction-validation/build_inputs/load_raw_data.py:48: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(txt_path, sep=';',infer_datetime_format=True)


In [7]:
df_subway_3min.head()

,COD_TRG,VAL_DATE,Flow
0,AMP,2020-02-01 00:00:00,9
1,AMP,2020-02-01 00:03:00,6
2,AMP,2020-02-01 00:06:00,3
3,AMP,2020-02-01 00:09:00,18
4,AMP,2020-02-01 00:12:00,13


In [10]:
df_tramway_15min.head()

,LIG_NUMERO_SAE,VAL_ARRET_CODE,VAL_DATE,Flow
0,0,0,2020-02-01 00:00:00,11
1,0,0,2020-02-01 00:15:00,9
2,0,0,2020-02-01 07:15:00,2
3,0,0,2020-02-01 07:30:00,5
4,0,0,2020-02-01 07:45:00,26


In [11]:
df_bus_15min.head()

,LIG_NUMERO_SAE,VAL_ARRET_CODE,VAL_DATE,Flow
0,0,0,2020-02-01 00:00:00,18
1,0,0,2020-02-01 00:15:00,4
2,0,0,2020-02-01 00:30:00,2
3,0,0,2020-02-01 01:30:00,14
4,0,0,2020-02-01 02:00:00,1


In [13]:
df_velov_30min.head()

,id_sortie,date_sortie,volume
0,169,2020-05-20 13:30:00,10
1,169,2020-05-20 14:00:00,19
2,169,2020-05-26 13:30:00,9
3,201,2019-01-04 11:00:00,1
4,201,2019-01-07 11:00:00,1


In [29]:
df_criter_6min_october.head()

,ID_POINT_MESURE,NOM_POINT_MESURE,DEBIT_HEURE,TAUX_HEURE,HORODATE,NOMBRE_ECH_1_MIN_MANQUANTS,day,str_hour_min,hour_min
0,1,Berthelot_Servant_Mace,380,3,2019-10-01 00:00:00,0,1,0:0,0.0
1,1,Berthelot_Servant_Mace,240,2,2019-10-01 00:06:00,0,1,0:6,1.0
2,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:12:00,0,1,0:12,2.0
3,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:18:00,0,1,0:18,3.0
4,1,Berthelot_Servant_Mace,200,1,2019-10-01 00:24:00,0,1,0:24,4.0


## Shapefile:

In [ ]:
import json 
import geopandas as gpd
from shapely.geometry import Point

## Velov 
html_save_path = 'velov_map.html'
json_name = 'locations/pvo_patrimoine_voirie.pvostationvelov.json'
stations_json = json.load(open(f"{ROOT_DATA}/raw_data/VELO-LYON/{json_name}",'rb'))

gdf_velov = gpd.GeoDataFrame(
    [
        {**feat['properties'],
         'geometry': Point(*feat['geometry']['coordinates'])}
        for feat in stations_json['features']
    ],
    crs="EPSG:4326"
)
## ----

## CRITER 
CRITER_PATH = f"{ROOT_DATA}/raw_data/Comptages_Velo_Routier/CRITER"
json_name = 'pvo_patrimoine_voirie.pvocomptagecriter.json'
sensor_json = json.load(open(f"{CRITER_PATH}/{json_name}",'rb'))
sensor_json['values']

gdf_criter = gpd.GeoDataFrame(
    [
        {**feat,
         'geometry': Point(feat['lon'],feat['lat'])}
        for feat in sensor_json['values']
    ],
    crs="EPSG:4326"
)
## ----

## Subway 
folder_path = f'{ROOT_DATA}/raw_data/keolis_data_2019-2020'
ref_subway = pd.read_csv(f"{folder_path}/ref_subway.csv").rename(columns = {'MEAN_X' : 'lon','MEAN_Y':'lat'})
ref_subway = ref_subway[['lon','lat','COD_TRG','LIB_STA_SIFO']]
## ---


## Tramway & bus 
ref_tram_bus = pd.read_csv(f"{folder_path}/ref_tram_bus.csv").rename(columns = {'IDT_PNT' : 'VAL_ARRET_CODE','COO_X_WGS84':'lon','COO_Y_WGS84':'lat'})
ref_tram_bus = ref_tram_bus[['lon','lat','NOM_PNT','VAL_ARRET_CODE']]

ref_lines = pd.read_csv(f"{folder_path}/ref_lines.csv").rename(columns = {'COD_SAE_PCC':'LIG_NUMERO_SAE'})
ref_lines = ref_lines[['LIG_NUMERO_SAE','COD_LIG_CLI']]
## ---


# Indications Générales:

### df_subway
# The column EMP_CEB contain the subway lane followed by the TRG (trigram) code of the subway station : 'VMMx-TRG'
# Where 'x' could be 'A','B','C' or 'D' and TRG is used as an id of subway station (could be 'BON' for Bonnevay, and so on ).

### df_tramway
# The column EMP_CEB contain the id of tramway station (VAL_ARRET_CODE) followed by some number that I can't indentify
# We can match the Tramway data to their lane with "LIG_NUMERO_SAE" and the df "ref_lines"

### Tramway 
# We can match spatially the Tramway data with VAL_ARRET_CODE and the dataframe "ref_tram_bus"
# We can match the Tramway data to their lane with "LIG_NUMERO_SAE" which return an id, associated to the lane T1,T2,T3,T4,T5 or T6 available on "ref_lines"

### Bus
# LIG_NUMERO_SAE represent the id of the bus lane. We can match it with COD_SAE_PCC in 'ref_lane'
# The bus lane is identified by 'LIG_NUMERO_SAE'. We can get client name ('COD_LIG_CLI') from 'ref_lines' (by matching the id with COD_SAE_PCC)


In [53]:
gdf_velov.head()

,idstation,nom,adresse1,adresse2,commune,numdansarrondissement,nbbornettes,stationbonus,pole,ouverte,achevement,gid,code_insee,geometry
0,7024,Bancel / Chevreul,Rue Bancel,Rue Chevreul,Lyon 7 ème,24.0,21.0,,Itinéraire cyclable,Oui,,1,69387,POINT (4.84052 45.74852)
1,10063,Perrin / Jean Jaurès,"9, Place Jules Grandclément",Rue Antonin Perrin,VILLEURBANNE,63.0,34.0,,quartier Grandclément,Oui,,2,69266,POINT (4.88561 45.75839)
2,10021,GrandClément,Place GrandClément,"171, Cours Tolstoï",VILLEURBANNE,21.0,19.0,,"centre quartier, futur arrêt C3, itinéraire cy...",Oui,,3,69266,POINT (4.88995 45.75972)
3,8038,Albert Thomas / Villon,"64, Cours Albert Thomas",angle rue Villon,Lyon 8 ème,38.0,20.0,,"bureaux, itinéraire cyclable",Oui,,4,69388,POINT (4.86743 45.74672)
4,7045,Parc des Berges,"Face 40, Avenue Leclerc",,Lyon 7 ème,45.0,25.0,,"Accès berges, parc, itinéraire cyclable",Oui,,5,69387,POINT (4.82449 45.73774)


In [54]:
gdf_criter.head()

,anneereference,debithorairemax,distancelignearret,estvalide,gid,horairedebitmax,identifiantcontroleur,identifiantptm,lat,lon,moyennejoursouvrable,nbvoies,nom,positionnement,typecapteur,typepostemesure,geometry
0,NaN,NaN,0,None,14498,None,779.0,1347.0,45.692988,4.946386,NaN,1.0,1347_Egalite_Colette_Rostand,1347_V,Capteur inductif Criter,Boucle inductive TV,POINT (4.94639 45.69299)
1,2020.0,5139.0,0,None,15437,07h00,-1.0,2106.0,45.668380,4.845563,55656.0,3.0,2106_Voie_A7_Feyzin_Paris,2106_VL,Capteur inductif Criter,Boucle inductive TV,POINT (4.84556 45.66838)
2,2020.0,96.0,0,None,13768,08h00,983.0,650.0,45.783764,4.831050,533.0,2.0,650_Velo_Canuts_SN,650_SN,Capteur inductif Criter,Boucle inductive Vélo,POINT (4.83105 45.78376)
3,2019.0,5934.0,0,None,15353,17h00,-1.0,1980.0,45.794817,4.779847,52269.0,3.0,1980_Voie_A6_Ecully_Paris_1,1980_VR,Capteur inductif Criter,Boucle inductive TV,POINT (4.77985 45.79482)
4,2020.0,723.0,108,None,14070,08h00,368.0,918.0,45.769988,4.812657,6646.0,1.0,918_Koenig_Chauveau_St Vincent,918_V,Capteur inductif Criter,Boucle inductive TV,POINT (4.81266 45.76999)


In [60]:
ref_subway.head()

,lon,lat,COD_TRG,LIB_STA_SIFO
0,4.829212,45.753114,AMP,AMPERE
1,4.833397,45.757263,BEL,BELLECOUR
2,4.859429,45.766892,BRO,BROTTEAUX
3,4.835813,45.763449,COR,CORDELIERS
4,4.832498,45.785258,CUI,CUIRE


In [59]:
ref_tram_bus.head()

,lon,lat,NOM_PNT,VAL_ARRET_CODE
0,4.804221,45.780508,Gare de Vaise,30093
1,4.805314,45.766451,Gorge de Loup,30094
2,4.826904,45.749805,Perrache.,30101
3,4.833019,45.757627,Bellecour,30103
4,4.835735,45.763428,Cordeliers-,30106


In [58]:
ref_lines.head()

,LIG_NUMERO_SAE,COD_LIG_CLI
0,304.0,D
1,303.0,C
2,326.0,F2
3,325.0,F1
4,530.0,T2
